In [ ]:
!pip install -q numpy-stl gdown

In [ ]:
from logging import currentframe
import numpy as np
from stl import mesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from matplotlib.colors import Normalize
from scipy.interpolate import griddata
import cv2 as cv
import pandas as pd

In [ ]:
!gdown --id 1_9NPC8E-kt4pU6rTi5lkh91eFR_jnFav
!unzip -q /kaggle/working/wb_2D3Dretrieval_dataset.zip
!rm /kaggle/working/wb_2D3Dretrieval_dataset.zip

In [ ]:
# Define path
path = '/kaggle/working/wb_2D3Dretrieval_dataset'
queries_path = os.path.join(path, 'queries')
database3D_2D_path = os.path.join(path, 'database_2D')


if not os.path.exists(database3D_2D_path):
    os.mkdir(database3D_2D_path)

for obj in os.listdir(os.path.join(path, 'database')):
    obj_name = obj.split('.')[0]

    your_mesh = mesh.Mesh.from_file(os.path.join(path, f'database/{obj}'))

    # Get center of gravity
#     cog = np.mean(your_mesh.vectors.reshape(-1, 3), axis=0)
    cog = [np.max(your_mesh.x) * 0.5, np.max(your_mesh.y) * 0.5, np.max(your_mesh.z) * 0.5]

    # Define the equation of the cutting plane
    normal = np.array([0, 0, 1])  # normal vector of the plane

    intersection_points = []
    for i in range(len(your_mesh.vectors)):
        for j in range(3):
            v0 = your_mesh.vectors[i, j - 1]
            v1 = your_mesh.vectors[i, j]
            if np.dot(normal, v0 - cog) * np.dot(normal, v1 - cog) < 0:
                t = np.dot(normal, cog - v0) / np.dot(normal, v1 - v0)
                intersection_points.append(v0 + t * (v1 - v0))

    # Convert intersection points to NumPy array
    above_plane_vertices = your_mesh.vectors[your_mesh.vectors[:, :, 2] > cog[2]]

    # Flatten the vertices and remove duplicates
    # intersection_points = np.unique(above_plane_vertices.reshape(-1, 3), axis=0)
    intersection_points = above_plane_vertices

    # Convert intersection points to NumPy array
    intersection_points = np.array(intersection_points)

    # Project 3D vertices onto 2D plane (discard z-coordinate)
    projection_points = intersection_points[:, :2]

    # Calculate grayscale values based on z-coordinate
    z_values = intersection_points[:, 2]
    try:
      min_z, max_z = np.min(z_values), np.max(z_values)
    except ValueError as e:
      print(obj)

    # Invert grayscale values
    inverted_values = max_z - z_values

    # Define grid for interpolation
    x_grid = np.linspace(min(projection_points[:, 0]), max(projection_points[:, 0]), 200)
    y_grid = np.linspace(min(projection_points[:, 1]), max(projection_points[:, 1]), 200)
    xx, yy = np.meshgrid(x_grid, y_grid)

    # Perform bilinear interpolation
    interpolated_values = griddata(projection_points, inverted_values, (xx, yy), method='linear')

    # Normalize interpolated values
    norm = Normalize(vmin=min_z, vmax=max_z)
    grayscale_values = norm(interpolated_values)

    plt.figure(figsize=(5.12, 5.12))

    rotated_grayscale_values = np.rot90(grayscale_values, k=2)  # Rotate 180 degrees

    flipped_grayscale_values = np.fliplr(rotated_grayscale_values)  # Horizontal flip

    plt.imshow(flipped_grayscale_values, extent=(min(x_grid), max(x_grid), min(y_grid), max(y_grid)), cmap='gray')
    plt.axis('off')

    plt.savefig(os.path.join(database3D_2D_path, f'/{obj_name}.png'))

    plt.close()

In [ ]:
bf = cv.BFMatcher()
sift = cv.SIFT_create()

queries_sift = []
target_sift = []

for x in os.listdir(queries_path):
    kp, des = sift.detectAndCompute(cv.imread(os.path.join(queries_path, x),cv.IMREAD_GRAYSCALE),None)
    queries_sift.append((x, (kp, des)))

for x in os.listdir(database3D_2D_path):
    kp, des = sift.detectAndCompute(cv.imread(os.path.join(database3D_2D_path, x),cv.IMREAD_GRAYSCALE),None)
    target_sift.append((x, (kp, des)))

def matcher(des1, des2):
    matches = bf.knnMatch(des1,des2,k=2)
    cnt = 0
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            cnt += 1
        
    return cnt

In [ ]:
query_mapping = {}

mrr_at_5 = 0

for file, (kp, des) in queries_sift:
    scores = [(matcher(des, d), f.split('.')[0] + '.stl') for f, (k, d) in target_sift]
    scores.sort(reverse=True)

    query_mapping[file] = ''
    for i in range(5):
        query_mapping[file] += scores[i][1] + ','
        if scores[i][1].split('.')[0] == file.split('.')[0]:
            mrr_at_5 += 1/ (i + 1)
    query_mapping[file] = query_mapping[file][:-1]
    
    print(file, query_mapping[file])

queries_len = len(os.listdir(os.path.join(query_path)))
print('MRR@5: ', mrr_at_5 / queries_len)

In [ ]:
output = {
    'query': query_mapping.keys(),
    'label': query_mapping.values()
}

df = pd.DataFrame.from_dict(output)
df.to_csv('predict.csv', sep=',', index=False)